In [21]:
import xgboost as xgb
print(xgb.__version__)

3.0.4


In [22]:
import sys, xgboost as xgb
print(sys.executable)        # should point to .../.venv/bin/python
print(xgb.__version__)       # should print 3.0.4
print(xgb.__file__)          # should live under .../.venv/...

/Users/kalpitpatel/CascadeProjects/housing-price-regressor/venv/bin/python
3.0.4
/Users/kalpitpatel/CascadeProjects/housing-price-regressor/venv/lib/python3.14/site-packages/xgboost/__init__.py


In [23]:
# ==============================================
# 1. Imports
# ==============================================
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from xgboost import XGBRegressor
import optuna
import mlflow
import mlflow.xgboost

In [24]:
# ==============================================
# 2. Load processed datasets
# ==============================================
train_df = pd.read_csv("/Users/kalpitpatel/CascadeProjects/housing-price-regressor/data/processed/feature_engineered_train.csv")
eval_df = pd.read_csv("/Users/kalpitpatel/CascadeProjects/housing-price-regressor/data/processed/feature_engineered_eval.csv")


# Define target + features
target = "price"
X_train, y_train = train_df.drop(columns=[target]), train_df[target]
X_eval, y_eval   = eval_df.drop(columns=[target]), eval_df[target]

print("Train shape:", X_train.shape)
print("Eval shape:", X_eval.shape)

Train shape: (576815, 39)
Eval shape: (148448, 39)


In [25]:
# ==============================================
# 3. Define Optuna objective function with MLflow
# ==============================================
def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 200, 1000),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        "gamma": trial.suggest_float("gamma", 0.0, 5.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 10.0, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 10.0, log=True),
        "random_state": 42,
        "n_jobs": -1,
        "tree_method": "hist",
    }

    with mlflow.start_run(nested=True):
        model = XGBRegressor(**params)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_eval)
        rmse = float(np.sqrt(mean_squared_error(y_eval, y_pred)))
        mae = float(mean_absolute_error(y_eval, y_pred))
        r2 = float(r2_score(y_eval, y_pred))

        # Log hyperparameters + metrics
        mlflow.log_params(params)
        mlflow.log_metrics({"rmse": rmse, "mae": mae, "r2": r2})

    return rmse

In [26]:
# ==============================================
# 4. Run Optuna study with MLflow
# ==============================================
# Force MLflow to always use the root project mlruns folder
mlflow.set_tracking_uri("/Users/kalpitpatel/CascadeProjects/housing-price-regressor/mlruns")
mlflow.set_experiment("xgboost_optuna_housing")

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=3)

print("Best params:", study.best_trial.params)

[I 2026-01-13 19:55:11,546] A new study created in memory with name: no-name-00638f8a-bb3d-42f7-bb82-48907863cb5d
[I 2026-01-13 19:55:20,978] Trial 0 finished with value: 76435.51204499477 and parameters: {'n_estimators': 550, 'max_depth': 6, 'learning_rate': 0.272823473896624, 'subsample': 0.6368010223833221, 'colsample_bytree': 0.9857648935515575, 'min_child_weight': 7, 'gamma': 4.461182526407829, 'reg_alpha': 0.3104548695267258, 'reg_lambda': 5.010832088618717e-07}. Best is trial 0 with value: 76435.51204499477.
[I 2026-01-13 19:55:29,293] Trial 1 finished with value: 68486.73185268659 and parameters: {'n_estimators': 419, 'max_depth': 7, 'learning_rate': 0.12079382175817863, 'subsample': 0.568189376491691, 'colsample_bytree': 0.6145059973990092, 'min_child_weight': 3, 'gamma': 2.9794177934726163, 'reg_alpha': 0.783486482386129, 'reg_lambda': 9.992976341161174e-05}. Best is trial 1 with value: 68486.73185268659.
[I 2026-01-13 19:55:36,137] Trial 2 finished with value: 75791.43650379

Best params: {'n_estimators': 419, 'max_depth': 7, 'learning_rate': 0.12079382175817863, 'subsample': 0.568189376491691, 'colsample_bytree': 0.6145059973990092, 'min_child_weight': 3, 'gamma': 2.9794177934726163, 'reg_alpha': 0.783486482386129, 'reg_lambda': 9.992976341161174e-05}


In [33]:
# ==============================================
# 5. Train final model with best params and log to MLflow
# ==============================================
best_params = study.best_trial.params
best_model = XGBRegressor(**best_params)
best_model.fit(X_train, y_train)

y_pred = best_model.predict(X_eval)

mae = mean_absolute_error(y_eval, y_pred)
rmse = np.sqrt(mean_squared_error(y_eval, y_pred))
r2 = r2_score(y_eval, y_pred)

print("Final tuned model performance:")
print("MAE:", mae)
print("RMSE:", rmse)
print("R²:", r2)

# Log final model
with mlflow.start_run(run_name="best_xgboost_model"):
    mlflow.log_params(best_params)
    mlflow.log_metrics({"rmse": rmse, "mae": mae, "r2": r2})
    best_model._estimator_type = "regressor"
    mlflow.xgboost.log_model(best_model, name="model")
    

Final tuned model performance:
MAE: 32742.51107480013
RMSE: 72570.52829380102
R²: 0.9593012940709754


2026/01/13 20:00:03 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /var/folders/06/vtzh798j5hv7bq6kv7q4sm480000gn/T/tmpryibfbam/model, flavor: xgboost). Fall back to return ['xgboost==3.0.4']. Set logging level to DEBUG to see the full traceback. 
